### <a id='top_of_page'>Dynamically Creating DataFrames</a>

This script demonstrates:
1. How to dynamically create dataframes from reading files, which are then stored into a dictionary
2. How to append each dataframe into a master dataframe using two different methods: pd.DataFrame.append() and pd.concat()
    - <i>** pd.DataFrame.append() was deprecated in pandas 1.4.0 onward</i>
    - Section links for these methods:
        - [Consolidate with pd.DataFrame.append()](#append)
        - [Consolidate with pd.concat()](#concat)

In [1]:
# Consolidate all imports here
import pandas as pd
import datetime as dt
import glob

In [2]:
# Version check pandas
print('pandas:', pd.__version__)

pandas: 1.3.4


In [3]:
# Define files
files = glob.glob('*.xlsx')
files

['file1.xlsx', 'file2.xlsx', 'file3.xlsx']

In [4]:
# Create dictionary of dataframes
master_dict = {}

# Loop to clean up files and create a dataframe for each file in the dictionary
for f in files:
    df = pd.read_excel(f)
    df = df.iloc[1:,2:].reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df.drop(df.index[0]).reset_index(drop=True)
    master_dict[f'{os.path.splitext(f)[0]}'] = df       # Splitext to extract filename without extension

In [5]:
# Check dictionary items
for key, data in master_dict.items():
    print(key)
    display(data)

file1


,File,Date
0,file1,04-22-22
1,file1,04-22-22
2,file1,04-22-22


file2


,File,Date
0,file2,04-23-22
1,file2,04-23-22
2,file2,04-23-22


file3


,File,Date
0,file3,04-24-22
1,file3,04-24-22
2,file3,04-24-22


#### <a id='append'>Consolidate Master DataFrame Using pd.DataFrame.append()</a>
<i>[** Go to top of page](#top_of_page)</i>

Can only use this method prior to pandas 1.4.0


In [6]:
# Create master dataframe and consolidate
df_master = pd.DataFrame()

for key, data in master_dict.items():
    df_master = df_master.append(pd.concat([data]))

# Final cleanup on consolidated dataframe
df_master = df_master.reset_index(drop=True)
df_master['Date'] = pd.to_datetime(df_master['Date'])
df_master

,File,Date
0,file1,2022-04-22
1,file1,2022-04-22
2,file1,2022-04-22
3,file2,2022-04-23
4,file2,2022-04-23
5,file2,2022-04-23
6,file3,2022-04-24
7,file3,2022-04-24
8,file3,2022-04-24


#### <a id='concat'>Consolidate Master DataFrame Using pd.concat method</a>

<i>[** Go to top of page](#top_of_page)</i>

Must use this method for pandas 1.4.0 and later

In [7]:
# Instantiate lists to append dictionary data
filename_list = []
filename_date_list = []

In [8]:
# Append dictionary data into lists
for key, data in master_dict.items():
    filename_list.append(data['File'])
    filename_date_list.append(data['Date'])

In [9]:
# Need to unnest the lists; store in separate list objects

filename_list_unnested = []
for i in filename_list:
    for j in i:
        filename_list_unnested.append(j)

filename_date_list_unnested = []
for i in filename_date_list:
    for j in i:
        filename_date_list_unnested.append(j)

In [10]:
# Validate list lengths
print(len(filename_list_unnested))
print(len(filename_date_list_unnested))

9
9


In [11]:
# Create master dataframe using unnested lists

df_master_2 = pd.DataFrame({
    'File': filename_list_unnested,
    'Date': filename_date_list_unnested
})

df_master_2

,File,Date
0,file1,04-22-22
1,file1,04-22-22
2,file1,04-22-22
3,file2,04-23-22
4,file2,04-23-22
5,file2,04-23-22
6,file3,04-24-22
7,file3,04-24-22
8,file3,04-24-22
